<a href="https://colab.research.google.com/github/Daniel-Fernandez-951/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Build-1/Daniel_Fernandez_DS20_BuildWeek_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Data Handling
import pandas as pd
import numpy as np
import plotly.express as px


In [3]:
# Get data from URL and output to known filename.
!wget "https://healthdata.gov/node/3376186/download" -O detailed_covid.csv

--2020-09-24 16:43:30--  https://healthdata.gov/node/3376186/download
Resolving healthdata.gov (healthdata.gov)... 54.243.212.26
Connecting to healthdata.gov (healthdata.gov)|54.243.212.26|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.cdc.gov/api/views/vbim-akqf/rows.csv?accessType=DOWNLOAD [following]
--2020-09-24 16:43:30--  https://data.cdc.gov/api/views/vbim-akqf/rows.csv?accessType=DOWNLOAD
Resolving data.cdc.gov (data.cdc.gov)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cdc.gov (data.cdc.gov)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘detailed_covid.csv’

detailed_covid.csv      [     <=>            ] 377.43M  5.35MB/s    in 74s     

2020-09-24 16:44:45 (5.09 MB/s) - ‘detailed_covid.csv’ saved [395761628]



In [66]:
# Read data, drop some columns.
data = pd.read_csv("/content/detailed_covid.csv")
data = data.drop(['icu_yn', 'hosp_yn', 'current_status', 'sex', 'cdc_report_dt','pos_spec_dt', 'onset_dt', 'Race and ethnicity (combined)'], axis=1)
# Commented out visual check (optional)
# data.head(20)
data_clean = pd.DataFrame.copy(data)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.



In [67]:
# Masks to observe unusable data in `death_yn`.
missing_mask = (data['death_yn'] == 'Missing')
unk_mask = (data['death_yn'] == 'Unknown')

# Ouput some stats on the removal process.
rm_percent = ((data.shape[0]) - ((data[missing_mask].shape[0]) + (data[unk_mask].shape[0]))) / data.shape[0]
print(f'Unknown = {data[unk_mask].shape[0]}\nMissing = {data[missing_mask].shape[0]}\n    Sum = {(data[missing_mask].shape[0]) + (data[unk_mask].shape[0])}')
print(f'\nPurged Data = {round(rm_percent*100, 1)}%')


# Make binary, for later counting and NaN for removal.
data_clean['death_01'] = data['death_yn'].map({'Yes': 1, 'No': 0, 'Missing': np.NaN, 'Unknown': np.NaN})


Unknown = 660056
Missing = 1231997
    Sum = 1892053

Purged Data = 48.3%


In [68]:
# Refrence binary column for NaN removal.
data_clean = data_clean.dropna(how='any', subset=['death_01'])
# Check the difference for a mismatch.
print(f'Cleaning Complete? {(data.shape[0] - data_clean.shape[0]) == (data[missing_mask].shape[0]) + (data[unk_mask].shape[0])}')

Cleaning Complete? True


In [69]:
# Combine `Unknown` and `NaN` to `Unknown Age`
data_clean['age_grp'] = data_clean['age_group'].replace(regex=['Unknown', np.nan], value='Unknown Age')
# data_clean['age_grp'].value_counts(dropna=False)

# If information is missing; safe to assume unknown.
data_clean['medcond_clean'] = data_clean['medcond_yn'].replace(regex='Missing', value='Unknown' )
# data_clean['medcond_clean'] = data_clean['medcond_yn'].map({'Missing': 'Unknown'})
# 960418
# (data_clean['medcond_clean'] == 'Unknown').value_counts()

In [70]:
# Multiaxis pivot table, Mortality and Comorbidity
data_clean = pd.DataFrame(data_clean)
table = data_clean.pivot_table(columns='age_grp', index=['death_yn', 'medcond_clean'], values='death_01', aggfunc='count' )
table

age_grp                 0 - 9 Years  10 - 19 Years  ...  80+ Years  Unknown Age
death_yn medcond_clean                              ...                        
No       No                   10467          26768  ...       1867           78
         Unknown              42353          94462  ...      45982          477
         Yes                   4306          11990  ...      18088           64
Yes      No                       8              4  ...        679            1
         Unknown                 25             39  ...      29029           11
         Yes                     11             29  ...      27476            8

[6 rows x 10 columns]

In [71]:
table2 = data_clean.pivot_table(index='age_grp', columns=['death_yn'], values='death_01', aggfunc='count' )
table2

death_yn,No,Yes
age_grp,,
0 - 9 Years,57126,44
10 - 19 Years,133220,72
20 - 29 Years,321116,562
30 - 39 Years,288368,1618
40 - 49 Years,266038,3973
50 - 59 Years,252840,10319
60 - 69 Years,171088,21262
70 - 79 Years,88050,30816
80+ Years,65937,57184


In [77]:
data_clean['medcond_clean_01'] = data_clean['medcond_clean'].map({'Yes': 1, 'No': 0})
table3 = data_clean.pivot_table(index='age_grp', columns=['medcond_clean'], values='medcond_clean_01', aggfunc='count' )
long_df2 = table3.reset_index().melt(id_vars='age_grp', value_name='medcond_clean_01')
long_df2

,age_grp,medcond_clean,medcond_clean_01
0,0 - 9 Years,No,10475
1,10 - 19 Years,No,26772
2,20 - 29 Years,No,60207
3,30 - 39 Years,No,46556
4,40 - 49 Years,No,37053
5,50 - 59 Years,No,29162
6,60 - 69 Years,No,14259
7,70 - 79 Years,No,4751
8,80+ Years,No,2546
9,Unknown Age,No,79


In [36]:
long_df = table2.reset_index().melt(id_vars="age_grp", value_name="death_01")
long_df

,age_grp,death_yn,death_01
0,0 - 9 Years,No,57126
1,10 - 19 Years,No,133220
2,20 - 29 Years,No,321116
3,30 - 39 Years,No,288368
4,40 - 49 Years,No,266038
5,50 - 59 Years,No,252840
6,60 - 69 Years,No,171088
7,70 - 79 Years,No,88050
8,80+ Years,No,65937
9,Unknown Age,No,619


In [73]:
fig = px.bar(long_df, x='age_grp', y='death_01', color='death_yn',
             labels={"age_grp": "Age Grouping", "death_01": "Count", "death_yn": "Mortality"})
fig.update_layout(
    font=dict(size=16, family="Courier New, monospace", color='black'),
    title={
        'text': "Survival & Mortality By Age",
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    xaxis_title="Age Range",
    yaxis_title="Count",
    hovermode="x",
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)'
    )
fig.update_traces(hovertemplate='Count: %{y} <br>Age Range: %{x}')
fig.show()

In [ ]:
fig = px.bar(long_df, x='age_grp', y='death_01', color='death_yn',
             labels={"age_grp": "Age Grouping", "death_01": "Count", "death_yn": "Mortality"})
fig.update_layout(
    font=dict(size=16, family="Courier New, monospace", color='black'),
    title={
        'text': "Survival & Mortality By Age",
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    xaxis_title="Age Range",
    yaxis_title="Count",
    hovermode="x",
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)'
    )
fig.update_traces(hovertemplate='Count: %{y} <br>Age Range: %{x}')
fig.show()

In [78]:
fig = px.line(long_df2, x='age_grp', y='medcond_clean_01')
fig.update_traces(mode="markers+lines")
fig.update_layout(
    font=dict(size=16, family="Courier New, monospace", color='black'),
    title={
        'text': "Disclosed Comorbidities By Age",
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top'
        },
    xaxis_title="Age Range",
    yaxis_title="Count",
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)'
      )
fig.update_traces(hovertemplate='%{y}')
fig.show()